# Attachment Protocol Example -Bob


In [26]:
%autoawait
import time
import asyncio
import base64

IPython autoawait is `on`, and set to use `asyncio`


#### Intialise Controller

In [27]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

#### Check if active connection exists

In [28]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")


Results :  [{'their_label': 'Alice', 'accept': 'manual', 'connection_id': '9c350b90-4312-4cc7-b072-7a261475c545', 'routing_state': 'none', 'state': 'active', 'initiator': 'self', 'invitation_mode': 'once', 'invitation_key': '89jbF3LWuCeUdE8TJzuMP5iVJaWeyGCKv2rfbdn2ZMBU', 'updated_at': '2020-09-10 12:09:20.414213Z', 'my_did': '6QZAPijnyMrQgGvnpb9x9m', 'their_did': '5UZRMryktBo8Nhzz2q6p2G', 'created_at': '2020-09-10 12:05:10.012591Z'}]
Connection : {'their_label': 'Alice', 'accept': 'manual', 'connection_id': '9c350b90-4312-4cc7-b072-7a261475c545', 'routing_state': 'none', 'state': 'active', 'initiator': 'self', 'invitation_mode': 'once', 'invitation_key': '89jbF3LWuCeUdE8TJzuMP5iVJaWeyGCKv2rfbdn2ZMBU', 'updated_at': '2020-09-10 12:09:20.414213Z', 'my_did': '6QZAPijnyMrQgGvnpb9x9m', 'their_did': '5UZRMryktBo8Nhzz2q6p2G', 'created_at': '2020-09-10 12:05:10.012591Z'}
Active Connection ID :  9c350b90-4312-4cc7-b072-7a261475c545


### Instantiate Listener

This is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In this instance the handler decodes the sent attachment and stores it. 

In [29]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    filename = payload['filename']
    base64_bytes = data.encode('ascii')
    text = base64.b64decode(base64_bytes)
    file_content = text.decode('ascii')
    filepath = f"received_files/{filename}"
    f = open(filepath,"w+")
    f.write(file_content)
    f.close()
    
#     data = payload['content'] 
#     asyncio.get_event_loop().create_task(om_controller.messaging.send_message(connection_id, " Received!"))
attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([attach_listener], defaults=True)


In [30]:
response = await om_controller.protocol.send_attachment(connection_id,data)

print(response)

NameError: name 'data' is not defined

Handle Attachment {'connection_id': '9c350b90-4312-4cc7-b072-7a261475c545', 'message_id': '10f7e2c2-e4bf-47c5-a267-387fb11f0b01', 'attachment_message': 'Here is the data you wanted', 'content': 'VGhpcyBpcyBhIHRlc3QgZmlsZSB0byBhdHRhY2gu', 'filename': 'test_file.txt', 'state': 'received'} 9c350b90-4312-4cc7-b072-7a261475c545
Attachment Received



# End of Tutorial

Be sure to terminate the controller so you can run another tutorial.


In [25]:
response = await om_controller.terminate()
print(response)


None
